In [1]:
# Automatically reload modules after executing each cell.
%load_ext autoreload
%autoreload 2

In [2]:
# General imports
import os
import tensorflow as tf
from scipy import stats

# Utility imports
from utils.losses import *
from utils.plotting import *
from utils.training import *

np.random.seed(666) # Need to do more to ensure data is the same across runs.

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # pick a number < 4 on ML4HEP; < 3 on Voltan 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Multivariate Normal

## $C$ Parametrization

### Case 0

#### BCE $C$ Parametrization

In [4]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':bce, 'd':2}
params_2 = {'loss':tanh_bce, 'd':2, 'output': 'linear'}
params_3 = {'loss':arctan_bce, 'd':2,  'output': 'linear'}

filestr = 'models/multivariate/c_bce/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/vertical/X_trn.npy')[:N]
y = np.load('data/mvn/vertical/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [5]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 

2022-08-09 00:05:48.458783: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-08-09 00:05:48.466067: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2994530000 Hz
2022-08-09 00:05:48.467232: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b29584f5b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-09 00:05:48.467261: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-09 00:05:48.584448: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b2958b94a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-08-09 00:05:48.584497: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 6000, Compute Capability 7.5
2022-08-09 00:05:48.586167: I 

0.6885470151901245 	 27	0.6885713934898376 	 17	0.6885311603546143 	 18	
1 0.6885283589363098 	 25	0.6885502338409424 	 21	0.6885198950767517 	 21	
2 0.6885567903518677 	 14	0.6885592341423035 	 15	0.6885572671890259 	 22	
3 0.6885191202163696 	 19	0.6885514855384827 	 18	0.6885626316070557 	 23	
4 0.6885315775871277 	 22	0.6885408759117126 	 21	0.6885489821434021 	 16	
5 0.6885285377502441 	 21	0.6886304020881653 	 19	0.6885332465171814 	 13	
6 0.6885623335838318 	 20	0.6885992288589478 	 16	0.6885470747947693 	 18	
7 0.6885645389556885 	 19	0.6885461211204529 	 16	0.6885204911231995 	 16	
8 0.6885437369346619 	 26	0.6885830163955688 	 18	0.6885474920272827 	 27	
9 0.6885592341423035 	 28	0.6885626912117004 	 19	0.6885442137718201 	 15	
10 0.6885312795639038 	 20	0.6885726451873779 	 17	0.6885569095611572 	 30	
11 0.6885637044906616 	 22	0.6885776519775391 	 14	0.688551127910614 	 22	
12 0.6885223984718323 	 20	0.6886037588119507 	 21	0.6885414719581604 	 18	
13 0.6885256767272949 	 1

#### MSE $C$ Parametrization

In [6]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':mse, 'd':2}
params_2 = {'loss':tanh_mse, 'd': 2, 'output': 'linear'}
params_3 = {'loss':arctan_mse, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mse/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/vertical/X_trn.npy')[:N]
y = np.load('data/mvn/vertical/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [7]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.24770337343215942 	 16	0.2477184683084488 	 32	0.24771112203598022 	 19	
1 0.24769702553749084 	 18	0.24771980941295624 	 18	0.24771440029144287 	 33	
2 0.24771392345428467 	 20	0.2477130889892578 	 16	0.24770836532115936 	 23	
3 0.24769820272922516 	 19	0.24773184955120087 	 17	0.24770203232765198 	 22	
4 0.24770548939704895 	 22	0.24771930277347565 	 16	0.24771535396575928 	 22	
5 0.24769960343837738 	 26	0.24773365259170532 	 16	0.247701495885849 	 16	
6 0.24770329892635345 	 26	0.24771547317504883 	 25	0.24771206080913544 	 18	
7 0.24770015478134155 	 17	0.24771243333816528 	 30	0.24769939482212067 	 25	
8 0.24770133197307587 	 16	0.2477308064699173 	 19	0.24770089983940125 	 20	
9 0.24772585928440094 	 17	0.2477264702320099 	 28	0.2477000504732132 	 23	
10 0.2477090060710907 	 18	0.24772168695926666 	 35	0.24769993126392365 	 17	
11 0.24770969152450562 	 23	0.24771814048290253 	 18	0.2477024346590042 	 26	
12 0.24771514534950256 	 30	0.24772027134895325 	 26	0.2477056980133056

#### MLC $C$ Parametrization

In [8]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':mlc, 'd': 2, 'output': 'relu'}
params_2 = {'loss':square_mlc, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_mlc, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mlc/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/vertical/X_trn.npy')[:N]
y = np.load('data/mvn/vertical/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [9]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 -0.009190496988594532 	 92	0.17284847795963287 	 47	-0.009203076362609863 	 38	
1 -0.008918407373130322 	 79	0.20604214072227478 	 100	-0.009221386164426804 	 22	
2 -0.008858585730195045 	 68	-0.00875590369105339 	 65	-0.009230096824467182 	 15	
3 -0.008998725563287735 	 84	-0.008818468078970909 	 57	-0.009223783388733864 	 20	
4 -0.008993384428322315 	 100	-0.008373040705919266 	 46	-0.009220673702657223 	 20	
5 -0.009046831168234348 	 96	-0.008584965020418167 	 90	-0.009218012914061546 	 25	
6 -0.00919260922819376 	 59	-0.008146489970386028 	 53	-0.009268974885344505 	 21	
7 -0.00893300212919712 	 90	-0.009187299758195877 	 73	-0.009267264045774937 	 24	
8 -0.008950134739279747 	 56	-0.00856847409158945 	 60	-0.009267664514482021 	 21	
9 -0.009057901799678802 	 100	-0.007902047596871853 	 47	-0.009261681698262691 	 19	
10 -0.009117459878325462 	 87	-0.008075244724750519 	 66	-0.009201186709105968 	 22	
11 -0.008868103846907616 	 52	-0.008979873731732368 	 98	-0.009219706989824772 	

#### SQR $C$ Parametrization

In [10]:
# Experiment parameters
num = 0
reps = 100

# Model parameters
params_1 = {'loss':sqr, 'd': 2,'output': 'relu'}
params_2 = {'loss':square_sqr, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_sqr, 'd': 2,'output': 'linear'}

filestr = 'models/multivariate/c_sqr/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/vertical/X_trn.npy')[:N]
y = np.load('data/mvn/vertical/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [11]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.9957435131072998 	 79	1.5486072301864624 	 38	0.9953938722610474 	 22	
1 0.9960359930992126 	 100	1.2358183860778809 	 44	0.995361864566803 	 26	
2 0.9959938526153564 	 100	2.2703680992126465 	 47	0.9953778982162476 	 26	
3 0.9960553646087646 	 100	30.545175552368164 	 16	0.9953689575195312 	 18	
4 0.9984935522079468 	 100	1.051087737083435 	 100	0.9953925013542175 	 17	
5 0.9956555962562561 	 100	1.4566794633865356 	 100	0.9953917860984802 	 23	
6 0.9977561235427856 	 100	3.4161605834960938 	 25	0.9953656792640686 	 37	
7 0.9964891076087952 	 100	12.326827049255371 	 35	0.9953688979148865 	 17	
8 0.9958018660545349 	 100	13.744051933288574 	 18	0.9953814744949341 	 19	
9 1.0010724067687988 	 100	0.9981642961502075 	 100	0.9954213500022888 	 28	
10 0.9960557222366333 	 100	13.170145988464355 	 25	0.9953543543815613 	 17	
11 1.0087254047393799 	 100	30.330116271972656 	 35	0.9953856468200684 	 19	
12 0.9970239400863647 	 100	1.4991612434387207 	 83	0.9954150915145874 	 17	
13 0.9977

### Case 1

#### BCE $C$ Parametrization

In [12]:
# Experiment parameters
num = 1
reps = 100

# Model parameters
params_1 = {'loss':bce, 'd':2}
params_2 = {'loss':tanh_bce, 'd':2, 'output': 'linear'}
params_3 = {'loss':arctan_bce, 'd':2,  'output': 'linear'}

filestr = 'models/multivariate/c_bce/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/slant/X_trn.npy')[:N]
y = np.load('data/mvn/slant/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [13]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.6884206533432007 	 22	0.6884567737579346 	 18	0.6883795261383057 	 17	
1 0.6884080171585083 	 19	0.688445508480072 	 13	0.68839430809021 	 13	
2 0.6884124279022217 	 14	0.6883843541145325 	 12	0.688391387462616 	 23	
3 0.6884028315544128 	 24	0.6884641051292419 	 40	0.6884119510650635 	 18	
4 0.6883746385574341 	 17	0.6884503960609436 	 16	0.6884272694587708 	 25	
5 0.6883856654167175 	 19	0.6884512901306152 	 17	0.688408374786377 	 14	
6 0.688412606716156 	 14	0.6884877681732178 	 15	0.6883812546730042 	 17	
7 0.6884161233901978 	 15	0.6884335279464722 	 13	0.6883851885795593 	 15	
8 0.6883798837661743 	 19	0.6884292364120483 	 21	0.688398003578186 	 16	
9 0.6883743405342102 	 14	0.6883862614631653 	 18	0.6884188652038574 	 16	
10 0.6883658766746521 	 16	0.6883894801139832 	 16	0.6884037256240845 	 17	
11 0.6884177327156067 	 12	0.6884632706642151 	 17	0.6884180903434753 	 16	
12 0.6883819103240967 	 17	0.6883998513221741 	 17	0.6883782744407654 	 21	
13 0.6884030699729919 	 17	0.

#### MSE $C$ Parametrization

In [14]:
# Experiment parameters
num = 1
reps = 100

# Model parameters
params_1 = {'loss':mse, 'd':2}
params_2 = {'loss':tanh_mse, 'd': 2, 'output': 'linear'}
params_3 = {'loss':arctan_mse, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mse/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/slant/X_trn.npy')[:N]
y = np.load('data/mvn/slant/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [15]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.24762634932994843 	 18	0.24763450026512146 	 18	0.24763000011444092 	 17	
1 0.24763871729373932 	 17	0.24765029549598694 	 20	0.24762803316116333 	 19	
2 0.24762868881225586 	 17	0.24764002859592438 	 16	0.24762192368507385 	 25	
3 0.24763190746307373 	 19	0.2476464956998825 	 17	0.24765436351299286 	 19	
4 0.24763284623622894 	 22	0.24764621257781982 	 16	0.24763081967830658 	 18	
5 0.24763093888759613 	 18	0.247660830616951 	 19	0.24764227867126465 	 19	
6 0.24762773513793945 	 21	0.2476663589477539 	 17	0.24762912094593048 	 25	
7 0.24763406813144684 	 19	0.2476559728384018 	 12	0.2476453185081482 	 17	
8 0.2476213276386261 	 17	0.2476613074541092 	 18	0.24762891232967377 	 15	
9 0.2476242333650589 	 17	0.24765637516975403 	 15	0.24762196838855743 	 23	
10 0.24763500690460205 	 22	0.24766728281974792 	 16	0.24762886762619019 	 21	
11 0.24763302505016327 	 21	0.2476435899734497 	 16	0.24763493239879608 	 17	
12 0.2476145625114441 	 16	0.2476435750722885 	 18	0.2476302534341812 	 

#### MLC $C$ Parametrization

In [16]:
# Experiment parameters
num = 1
reps = 100

# Model parameters
params_1 = {'loss':mlc, 'd': 2, 'output': 'relu'}
params_2 = {'loss':square_mlc, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_mlc, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mlc/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/slant/X_trn.npy')[:N]
y = np.load('data/mvn/slant/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [17]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 -0.009303566999733448 	 58	-0.008808939717710018 	 88	-0.009585613384842873 	 18	
1 -0.00953054428100586 	 100	-0.009305872954428196 	 80	-0.009593911468982697 	 14	
2 -0.00950920395553112 	 89	-0.008236289955675602 	 100	-0.00942194927483797 	 19	
3 -0.009436432272195816 	 100	-0.007597730960696936 	 100	-0.00960451178252697 	 13	
4 -0.009258733130991459 	 100	-0.009195888414978981 	 65	-0.009582910686731339 	 15	
5 -0.009388119913637638 	 85	-0.009181304834783077 	 95	-0.00952999573200941 	 24	
6 -0.009379246272146702 	 80	-0.009026296436786652 	 93	-0.009524406865239143 	 14	
7 -0.009456461295485497 	 73	-0.008582529611885548 	 65	-0.009581876918673515 	 17	
8 -0.009471148252487183 	 80	-0.009116999804973602 	 71	-0.009551812894642353 	 19	
9 -0.009493361227214336 	 100	-0.007804015185683966 	 100	-0.009569089859724045 	 14	
10 -0.00938301533460617 	 74	-0.008720039390027523 	 80	-0.009578865952789783 	 14	
11 -0.00936673954129219 	 56	-0.008868275210261345 	 94	-0.009571105241775

#### SQR $C$ Parametrization

In [18]:
# Experiment parameters
num = 1
reps = 100

# Model parameters
params_1 = {'loss':sqr, 'd': 2,'output': 'relu'}
params_2 = {'loss':square_sqr, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_sqr, 'd': 2,'output': 'linear'}

filestr = 'models/multivariate/c_sqr/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/slant/X_trn.npy')[:N]
y = np.load('data/mvn/slant/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [19]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.9955137372016907 	 100	1.8983588218688965 	 24	0.9952322840690613 	 15	
1 0.996588945388794 	 100	13.28265380859375 	 22	0.9952554702758789 	 17	
2 0.9961406588554382 	 100	1.2593662738800049 	 86	0.9952191114425659 	 17	
3 0.9959563612937927 	 100	15.416947364807129 	 26	0.9952128529548645 	 15	
4 0.9958835244178772 	 100	58.263832092285156 	 16	0.9952071905136108 	 17	
5 0.995654284954071 	 100	1.7406761646270752 	 49	0.9952372908592224 	 17	
6 0.9972612261772156 	 100	1.0574482679367065 	 100	0.9952226877212524 	 24	
7 0.9986500144004822 	 100	10.57059383392334 	 12	0.9952302575111389 	 14	
8 0.9957670569419861 	 100	1.425384521484375 	 100	0.9952166080474854 	 22	
9 0.9967422485351562 	 100	1.7384119033813477 	 42	0.9952090978622437 	 15	
10 0.9959486126899719 	 100	6.148698806762695 	 23	0.9952225089073181 	 15	
11 0.9961681365966797 	 100	5.906076431274414 	 29	0.9952300190925598 	 32	
12 0.9955332279205322 	 100	1.1913583278656006 	 100	0.9952034950256348 	 16	
13 0.99645990

### Case 2

#### BCE $C$ Parametrization

In [20]:
# Experiment parameters
num = 2
reps = 100

# Model parameters
params_1 = {'loss':bce, 'd':2}
params_2 = {'loss':tanh_bce, 'd':2, 'output': 'linear'}
params_3 = {'loss':arctan_bce, 'd':2,  'output': 'linear'}

filestr = 'models/multivariate/c_bce/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/circle/X_trn.npy')[:N]
y = np.load('data/mvn/circle/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [21]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.6375991702079773 	 22	0.6387968063354492 	 22	0.6384098529815674 	 29	
1 0.6389102339744568 	 17	0.6382179856300354 	 21	0.6381881833076477 	 46	
2 0.6376977562904358 	 19	0.6384596824645996 	 14	0.6384581327438354 	 25	
3 0.6382322907447815 	 27	0.638814389705658 	 18	0.638238251209259 	 45	
4 0.6376962661743164 	 21	0.6388468742370605 	 22	0.6378863453865051 	 56	
5 0.6378346085548401 	 16	0.6386379599571228 	 27	0.6382549405097961 	 51	
6 0.6382488012313843 	 21	0.6379761099815369 	 16	0.6387396454811096 	 30	
7 0.6383816599845886 	 19	0.6392828226089478 	 19	0.6376904249191284 	 93	
8 0.6374841332435608 	 21	0.6392148733139038 	 18	0.6379358172416687 	 46	
9 0.6387103796005249 	 19	0.6383041143417358 	 27	0.638164222240448 	 53	
10 0.637263298034668 	 20	0.6385257244110107 	 15	0.6377780437469482 	 54	
11 0.6383420825004578 	 20	0.6385419964790344 	 20	0.6382705569267273 	 30	
12 0.638032853603363 	 20	0.6389197111129761 	 21	0.6376610398292542 	 54	
13 0.6380324363708496 	 24	

#### MSE $C$ Parametrization

In [22]:
# Experiment parameters
num = 2
reps = 100

# Model parameters
params_1 = {'loss':mse, 'd':2}
params_2 = {'loss':tanh_mse, 'd': 2, 'output': 'linear'}
params_3 = {'loss':arctan_mse, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mse/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/circle/X_trn.npy')[:N]
y = np.load('data/mvn/circle/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [23]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.22471477091312408 	 25	0.22498778998851776 	 22	0.22469693422317505 	 28	
1 0.2245859056711197 	 21	0.22495393455028534 	 15	0.22470873594284058 	 33	
2 0.22485674917697906 	 25	0.2251596599817276 	 19	0.2245519757270813 	 45	
3 0.22442440688610077 	 21	0.22486624121665955 	 16	0.22489401698112488 	 34	
4 0.22475869953632355 	 22	0.22474394738674164 	 18	0.22479474544525146 	 29	
5 0.2247912436723709 	 17	0.225042462348938 	 17	0.224702849984169 	 20	
6 0.22514191269874573 	 20	0.22487391531467438 	 15	0.22469311952590942 	 63	
7 0.2245991826057434 	 23	0.22481118142604828 	 23	0.22481049597263336 	 22	
8 0.22457775473594666 	 20	0.22482606768608093 	 17	0.22509083151817322 	 19	
9 0.22463908791542053 	 27	0.22485120594501495 	 20	0.22469280660152435 	 43	
10 0.22482830286026 	 17	0.22468432784080505 	 20	0.2246881127357483 	 24	
11 0.22486013174057007 	 22	0.22494062781333923 	 16	0.22488859295845032 	 16	
12 0.2243957221508026 	 20	0.22490733861923218 	 15	0.22480745613574982 	 3

#### MLC $C$ Parametrization

In [24]:
# Experiment parameters
num = 2
reps = 100

# Model parameters
params_1 = {'loss':mlc, 'd': 2, 'output': 'relu'}
params_2 = {'loss':square_mlc, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_mlc, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mlc/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/circle/X_trn.npy')[:N]
y = np.load('data/mvn/circle/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [25]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 -0.08748206496238708 	 61	-0.100953109562397 	 54	-0.1002427414059639 	 23	
1 -0.09401405602693558 	 46	-0.10122635960578918 	 38	-0.0991731584072113 	 18	
2 -0.10043571144342422 	 52	-0.10034587234258652 	 66	-0.09956326335668564 	 24	
3 -0.09863065928220749 	 48	-0.09997496753931046 	 69	-0.10068771243095398 	 26	
4 -0.10093122720718384 	 87	-0.09988253563642502 	 44	-0.09964857995510101 	 25	
5 -0.09979191422462463 	 97	-0.10077815502882004 	 42	-0.09962346404790878 	 17	
6 -0.101109080016613 	 54	-0.09988363832235336 	 43	-0.09974443167448044 	 22	
7 -0.099040687084198 	 62	-0.10053001344203949 	 39	-0.10035551339387894 	 21	
8 -0.09602426737546921 	 39	0.25420281291007996 	 50	-0.09882953017950058 	 21	
9 -0.10059406608343124 	 33	-0.09987861663103104 	 48	-0.09966929256916046 	 25	
10 -0.10014158487319946 	 93	-0.09984182566404343 	 77	-0.10022570937871933 	 25	
11 -0.10023287683725357 	 100	-0.0996914878487587 	 53	-0.09855061024427414 	 22	
12 -0.10063823312520981 	 48	-0.100

#### SQR $C$ Parametrization

In [26]:
# Experiment parameters
num = 2
reps = 100

# Model parameters
params_1 = {'loss':sqr, 'd': 2,'output': 'relu'}
params_2 = {'loss':square_sqr, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_sqr, 'd': 2,'output': 'linear'}

filestr = 'models/multivariate/c_sqr/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/circle/X_trn.npy')[:N]
y = np.load('data/mvn/circle/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [27]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.9738723635673523 	 86	1.6998060941696167 	 74	0.9412647485733032 	 18	
1 0.9513671398162842 	 100	2.704026460647583 	 33	0.9416758418083191 	 19	
2 0.9593828916549683 	 94	12.49216365814209 	 30	0.9418567419052124 	 23	
3 0.9888821244239807 	 100	2.6627113819122314 	 28	0.9415243268013 	 18	
4 0.9465246796607971 	 53	0.9572681188583374 	 100	0.9419073462486267 	 31	
5 0.9562798738479614 	 100	0.9495202302932739 	 100	0.941809356212616 	 21	
6 0.9651580452919006 	 77	27.73531150817871 	 11	0.9413242936134338 	 23	
7 0.970992386341095 	 100	19.237428665161133 	 19	0.9421420097351074 	 20	
8 0.9612560272216797 	 100	18.546390533447266 	 39	0.9409353733062744 	 21	
9 0.9543963670730591 	 79	11.627732276916504 	 48	0.9414504766464233 	 26	
10 0.948737382888794 	 45	1.402919888496399 	 100	0.9410040378570557 	 30	
11 0.9679149985313416 	 100	16.415233612060547 	 21	0.941003143787384 	 21	
12 0.9622605443000793 	 100	1.0434024333953857 	 100	0.9412407279014587 	 21	
13 0.950169026851654 	

### Case 3

#### BCE $C$ Parametrization

In [28]:
# Experiment parameters
num = 3
reps = 100

# Model parameters
params_1 = {'loss':bce, 'd':2}
params_2 = {'loss':tanh_bce, 'd':2, 'output': 'linear'}
params_3 = {'loss':arctan_bce, 'd':2,  'output': 'linear'}

filestr = 'models/multivariate/c_bce/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/hyperbola/X_trn.npy')[:N]
y = np.load('data/mvn/hyperbola/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [29]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.6372013688087463 	 37	0.6373575329780579 	 29	0.6372748613357544 	 89	
1 0.637156069278717 	 44	0.6374432444572449 	 35	0.6372637748718262 	 83	
2 0.6372053027153015 	 50	0.6377361416816711 	 30	0.6372775435447693 	 88	
3 0.6371131539344788 	 57	0.6372700333595276 	 32	0.637350857257843 	 100	
4 0.6370262503623962 	 40	0.6376225352287292 	 27	0.6373274326324463 	 72	
5 0.6372537612915039 	 44	0.6373728513717651 	 30	0.6373008489608765 	 81	
6 0.6369861364364624 	 38	0.6371206641197205 	 31	0.637458324432373 	 65	
7 0.6371001601219177 	 45	0.6373956203460693 	 30	0.6373626589775085 	 82	
8 0.6370185017585754 	 38	0.6372502446174622 	 41	0.6373214721679688 	 85	
9 0.6371196508407593 	 44	0.6373546719551086 	 33	0.6372664570808411 	 71	
10 0.6370599269866943 	 52	0.637390673160553 	 38	0.6374421119689941 	 62	
11 0.6371949911117554 	 39	0.6375337243080139 	 36	0.6373676657676697 	 92	
12 0.6371041536331177 	 49	0.637324333190918 	 28	0.6374568343162537 	 82	
13 0.6371301412582397 	 45

#### MSE $C$ Parametrization

In [30]:
# Experiment parameters
num = 3
reps = 100

# Model parameters
params_1 = {'loss':mse, 'd':2}
params_2 = {'loss':tanh_mse, 'd': 2, 'output': 'linear'}
params_3 = {'loss':arctan_mse, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mse/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/hyperbola/X_trn.npy')[:N]
y = np.load('data/mvn/hyperbola/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [31]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.22455792129039764 	 50	0.22458112239837646 	 32	0.22461333870887756 	 57	
1 0.22454161942005157 	 46	0.224595308303833 	 34	0.22462008893489838 	 63	
2 0.22453930974006653 	 56	0.2245655655860901 	 32	0.2245747298002243 	 60	
3 0.22454090416431427 	 55	0.22455529868602753 	 29	0.22461184859275818 	 64	
4 0.22454003989696503 	 52	0.224584698677063 	 28	0.22461798787117004 	 53	
5 0.22455012798309326 	 48	0.22458256781101227 	 30	0.22459353506565094 	 58	
6 0.2245217114686966 	 48	0.22458921372890472 	 40	0.22461998462677002 	 55	
7 0.2245517075061798 	 37	0.22463476657867432 	 37	0.22455543279647827 	 77	
8 0.22451113164424896 	 50	0.22457562386989594 	 38	0.22459128499031067 	 65	
9 0.2245505005121231 	 53	0.22461675107479095 	 35	0.22459475696086884 	 61	
10 0.22454358637332916 	 34	0.22462357580661774 	 32	0.2246244102716446 	 52	
11 0.2245555967092514 	 45	0.22459255158901215 	 35	0.22461752593517303 	 61	
12 0.22455577552318573 	 49	0.22461499273777008 	 29	0.22463349997997284 

#### MLC $C$ Parametrization

In [32]:
# Experiment parameters
num = 3
reps = 100

# Model parameters
params_1 = {'loss':mlc, 'd': 2, 'output': 'relu'}
params_2 = {'loss':square_mlc, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_mlc, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mlc/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/hyperbola/X_trn.npy')[:N]
y = np.load('data/mvn/hyperbola/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [33]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 -0.12514829635620117 	 100	-0.1155824214220047 	 100	-0.1284940540790558 	 59	
1 -0.11862123757600784 	 100	-0.12128297984600067 	 100	-0.1262609213590622 	 43	
2 -0.12130200862884521 	 100	-0.1219322606921196 	 100	-0.12810085713863373 	 48	
3 -0.11552688479423523 	 100	0.43787893652915955 	 34	-0.12801401317119598 	 57	
4 -0.12286506593227386 	 100	-0.12314239889383316 	 100	-0.12743613123893738 	 48	
5 -0.11870314925909042 	 100	0.16019566357135773 	 61	-0.1286383867263794 	 51	
6 -0.11945732682943344 	 100	-0.1148562803864479 	 74	-0.12759505212306976 	 54	
7 -0.12058676779270172 	 100	-0.12122173607349396 	 100	-0.12779289484024048 	 44	
8 -0.11609610915184021 	 100	-0.1131080836057663 	 63	-0.1268879771232605 	 29	
9 -0.1252584010362625 	 100	-0.12365727871656418 	 100	-0.12718354165554047 	 35	
10 -0.11427243798971176 	 100	-0.12318696826696396 	 100	-0.12779515981674194 	 46	
11 -0.1180616095662117 	 100	-0.1032957211136818 	 100	-0.12778490781784058 	 38	
12 -0.1114882230758

#### SQR $C$ Parametrization

In [34]:
# Experiment parameters
num = 3
reps = 100

# Model parameters
params_1 = {'loss':sqr, 'd': 2,'output': 'relu'}
params_2 = {'loss':square_sqr, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_sqr, 'd': 2,'output': 'linear'}

filestr = 'models/multivariate/c_sqr/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/hyperbola/X_trn.npy')[:N]
y = np.load('data/mvn/hyperbola/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [35]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.9663444757461548 	 75	0.9615768790245056 	 100	0.940159022808075 	 47	
1 0.9800189733505249 	 100	57.210060119628906 	 20	0.9402481317520142 	 43	
2 0.9650946259498596 	 97	5.9982709884643555 	 12	0.9402388334274292 	 46	
3 0.9632598757743835 	 100	0.9848616719245911 	 100	0.9401288628578186 	 41	
4 0.9661455154418945 	 100	0.9517080187797546 	 100	0.9401909708976746 	 43	
5 0.9686636328697205 	 100	1.8636040687561035 	 43	0.9402859807014465 	 40	
6 0.9529316425323486 	 71	27.79072380065918 	 16	0.9404811263084412 	 56	
7 0.9578327536582947 	 63	0.9895090460777283 	 100	0.9403101205825806 	 46	
8 0.9578631520271301 	 100	1.1510709524154663 	 100	0.9404174089431763 	 48	
9 0.9566292762756348 	 97	9.125247955322266 	 28	0.9401895999908447 	 45	
10 0.9586842656135559 	 100	1.19344961643219 	 100	0.9401663541793823 	 45	
11 0.9574147462844849 	 63	12.450742721557617 	 29	0.9402682781219482 	 47	
12 0.9524551033973694 	 100	14.815110206604004 	 12	0.9402095675468445 	 43	
13 0.960351645

### Case 4

#### BCE $C$ Parametrization

In [36]:
# Experiment parameters
num = 4
reps = 100

# Model parameters
params_1 = {'loss':bce, 'd':2}
params_2 = {'loss':tanh_bce, 'd':2, 'output': 'linear'}
params_3 = {'loss':arctan_bce, 'd':2,  'output': 'linear'}

filestr = 'models/multivariate/c_bce/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/checker/X_trn.npy')[:N]
y = np.load('data/mvn/checker/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [37]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.6757377982139587 	 32	0.6758279800415039 	 37	0.6756110191345215 	 42	
1 0.6757012009620667 	 32	0.6759417653083801 	 36	0.6756903529167175 	 38	
2 0.6755766868591309 	 27	0.675712525844574 	 41	0.6757351756095886 	 29	
3 0.6756595969200134 	 44	0.6756003499031067 	 27	0.6754989624023438 	 47	
4 0.6755942702293396 	 38	0.6758435964584351 	 31	0.67557293176651 	 59	
5 0.6756316423416138 	 38	0.6757358312606812 	 23	0.6755746603012085 	 43	
6 0.675844132900238 	 39	0.6757250428199768 	 39	0.6756071448326111 	 34	
7 0.675663948059082 	 45	0.6757137775421143 	 33	0.6756351590156555 	 55	
8 0.6757538914680481 	 40	0.6757538914680481 	 23	0.6757726073265076 	 35	
9 0.6755731105804443 	 30	0.6757517457008362 	 30	0.675609827041626 	 47	
10 0.6756536364555359 	 30	0.6757767796516418 	 35	0.6757218837738037 	 58	
11 0.6756325960159302 	 30	0.6759763956069946 	 27	0.6756150126457214 	 45	
12 0.6755447387695312 	 38	0.6758225560188293 	 28	0.6756325960159302 	 43	
13 0.6754755973815918 	 33	0

#### MSE $C$ Parametrization

In [38]:
# Experiment parameters
num = 4
reps = 100

# Model parameters
params_1 = {'loss':mse, 'd':2}
params_2 = {'loss':tanh_mse, 'd': 2, 'output': 'linear'}
params_3 = {'loss':arctan_mse, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mse/set_{}/'.format(num)
filestr_1 = filestr + 'sigmoid/model_{}.h5'
filestr_2 = filestr + 'tanh/model_{}.h5'
filestr_3 = filestr + 'arctan/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'sigmoid/'):
    os.mkdir(filestr + 'sigmoid/')
    
if not os.path.isdir(filestr + 'tanh/'):
    os.mkdir(filestr + 'tanh/')

if not os.path.isdir(filestr + 'arctan/'):
    os.mkdir(filestr + 'arctan/')

# Data parameters
N = 10**6
X = np.load('data/mvn/checker/X_trn.npy')[:N]
y = np.load('data/mvn/checker/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [39]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.2415860891342163 	 35	0.24156421422958374 	 41	0.24156734347343445 	 32	
1 0.2415250837802887 	 42	0.24161536991596222 	 26	0.24156346917152405 	 51	
2 0.24157950282096863 	 34	0.24158045649528503 	 29	0.24158313870429993 	 48	
3 0.2415255457162857 	 29	0.24159656465053558 	 33	0.24163052439689636 	 36	
4 0.24156059324741364 	 36	0.24155384302139282 	 34	0.24161429703235626 	 41	
5 0.24157461524009705 	 32	0.24163056910037994 	 26	0.24151872098445892 	 38	
6 0.24154922366142273 	 37	0.24162055552005768 	 33	0.24160024523735046 	 40	
7 0.24154052138328552 	 38	0.24158094823360443 	 28	0.24157677590847015 	 33	
8 0.24161884188652039 	 29	0.241596519947052 	 26	0.24154190719127655 	 37	
9 0.241561159491539 	 36	0.2416245937347412 	 29	0.2415534257888794 	 33	
10 0.24154064059257507 	 24	0.2416551411151886 	 22	0.24161481857299805 	 52	
11 0.24155083298683167 	 30	0.24168743193149567 	 31	0.24159163236618042 	 28	
12 0.24157510697841644 	 28	0.24163074791431427 	 37	0.24157807230949402

#### MLC $C$ Parametrization

In [40]:
# Experiment parameters
num = 4
reps = 100

# Model parameters
params_1 = {'loss':mlc, 'd': 2, 'output': 'relu'}
params_2 = {'loss':square_mlc, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_mlc, 'd': 2, 'output': 'linear'}

filestr = 'models/multivariate/c_mlc/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/checker/X_trn.npy')[:N]
y = np.load('data/mvn/checker/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [41]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 -0.034858521074056625 	 100	-0.034778863191604614 	 98	-0.036913029849529266 	 26	
1 -0.03556973487138748 	 100	-0.035460203886032104 	 100	-0.0366712361574173 	 40	
2 -0.03505413606762886 	 100	-0.035522326827049255 	 68	-0.03668936714529991 	 30	
3 -0.035295769572257996 	 100	0.5157005786895752 	 45	-0.03685227409005165 	 28	
4 -0.03494444862008095 	 80	-0.03591349720954895 	 53	-0.036643948405981064 	 33	
5 -0.03539449721574783 	 100	-0.03607042133808136 	 85	-0.03625098615884781 	 35	
6 -0.03509342670440674 	 100	-0.03566725179553032 	 53	-0.03589696064591408 	 28	
7 -0.0353098101913929 	 100	-0.036115195602178574 	 47	-0.036494180560112 	 37	
8 -0.03236877918243408 	 100	-0.03594180569052696 	 75	-0.036312565207481384 	 26	
9 -0.034863393753767014 	 100	-0.035780202597379684 	 68	-0.03655991330742836 	 35	
10 -0.03560306504368782 	 100	-0.03523845970630646 	 55	-0.036662422120571136 	 30	
11 -0.035628560930490494 	 100	-0.033005330711603165 	 100	-0.03691541776061058 	 30	
12 -0

#### SQR $C$ Parametrization

In [42]:
# Experiment parameters
num = 4
reps = 100

# Model parameters
params_1 = {'loss':sqr, 'd': 2,'output': 'relu'}
params_2 = {'loss':square_sqr, 'd': 2, 'output': 'linear'}
params_3 = {'loss':exp_sqr, 'd': 2,'output': 'linear'}

filestr = 'models/multivariate/c_sqr/set_{}/'.format(num)
filestr_1 = filestr + 'relu/model_{}.h5'
filestr_2 = filestr + 'square/model_{}.h5'
filestr_3 = filestr + 'exponential/model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

if not os.path.isdir(filestr + 'relu/'):
    os.mkdir(filestr + 'relu/')
    
if not os.path.isdir(filestr + 'square/'):
    os.mkdir(filestr + 'square/')

if not os.path.isdir(filestr + 'exponential/'):
    os.mkdir(filestr + 'exponential/')

# Data parameters
N = 10**6
X = np.load('data/mvn/checker/X_trn.npy')[:N]
y = np.load('data/mvn/checker/y_trn.npy')[:N]
data, m, s = split_data(X, y)

In [43]:
for i in range(reps):
    print(i, end = ' ')
    model_1, trace = train(data, **params_1)
    model_2, trace = train(data, **params_2)
    model_3, trace = train(data, **params_3)
    print()
    model_1.save_weights(filestr_1.format(i))
    model_2.save_weights(filestr_2.format(i))
    model_3.save_weights(filestr_3.format(i))

0 0.9906003475189209 	 100	0.9915164709091187 	 100	0.9820156097412109 	 29	
1 0.9846400022506714 	 100	1.5672199726104736 	 100	0.9819976091384888 	 30	
2 0.9849245548248291 	 100	1.0298744440078735 	 100	0.9820153713226318 	 42	
3 0.9881147742271423 	 100	1.6729146242141724 	 73	0.9821956157684326 	 34	
4 0.9901266098022461 	 100	1.0521339178085327 	 100	0.9821828007698059 	 30	
5 0.9870797991752625 	 100	1.089711308479309 	 64	0.9820052981376648 	 30	
6 0.9884868264198303 	 100	0.9889800548553467 	 100	0.9822700023651123 	 39	
7 0.9827427864074707 	 71	13.034248352050781 	 34	0.981997013092041 	 26	
8 0.9907568097114563 	 100	1.0495232343673706 	 100	0.9820417761802673 	 44	
9 0.9879547357559204 	 100	0.9862876534461975 	 100	0.9819488525390625 	 34	
10 0.9863759875297546 	 100	2.2505459785461426 	 52	0.9820910096168518 	 43	
11 0.9899349212646484 	 100	0.9968087077140808 	 100	0.9820531010627747 	 39	
12 0.9886042475700378 	 100	20.597339630126953 	 14	0.9822128415107727 	 28	
13 0

## $A/B$ Parametrization

See `sqr_ab.py` and `mse_ab.py`.

### $p^*$-MSE Models

#### Case 0 (Vertical)

In [15]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

num = 5    # vertical p*
#num = 6    # slant p*
#num = 7    # circle p*
#num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_mse/set_{}/'.format(num)
mse_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/vertical/X_trn.npy')[:N]
y = np.load('data/mvn/vertical/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [16]:
p_star = 1.12
params = {'loss': get_mse(p_star), 'd': 2}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(mse_filestr.format(i))
    print()

0.4443073868751526 	 43	0.44429343938827515 	 40	0.4443179965019226 	 36	0.4442988634109497 	 41	0.44431135058403015 	 51	0.4442954957485199 	 51	0.44429081678390503 	 58	0.4442935287952423 	 42	0.4443061947822571 	 41	0.4443158209323883 	 43	0.44430041313171387 	 38	0.4443006217479706 	 43	0.444308876991272 	 43	0.4443019926548004 	 50	0.44430267810821533 	 45	0.44430702924728394 	 53	0.4442935287952423 	 46	0.44429999589920044 	 41	0.4443104863166809 	 54	0.44430431723594666 	 35	0.4443162977695465 	 43	0.4443199336528778 	 38	0.44429299235343933 	 36	0.4442901611328125 	 45	0.4443131387233734 	 39	0.4443001449108124 	 31	0.4443000257015228 	 32	0.4443110525608063 	 42	0.44430363178253174 	 41	0.4443114995956421 	 44	0.4443056285381317 	 33	0.4443090856075287 	 43	0.44430646300315857 	 50	0.44431769847869873 	 51	0.44431817531585693 	 50	0.44430112838745117 	 28	0.4443025588989258 	 32	0.44431084394454956 	 41	0.4443066418170929 	 40	0.44429802894592285 	 42	0.44432172179222107 	 52	

#### Case 1 (Slant)

In [17]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
num = 6    # slant p*
#num = 7    # circle p*
#num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_mse/set_{}/'.format(num)
mse_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/slant/X_trn.npy')[:N]
y = np.load('data/mvn/slant/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [18]:
p_star = 1.16
params = {'loss': get_mse(p_star), 'd': 2}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(mse_filestr.format(i))
    print()

0.43404361605644226 	 36	0.4340241253376007 	 26	0.43403318524360657 	 45	0.43404099345207214 	 32	0.4340333044528961 	 29	0.4340304434299469 	 31	0.43402838706970215 	 35	0.4340236186981201 	 38	0.43402397632598877 	 24	0.43402180075645447 	 24	0.43403246998786926 	 25	0.43401646614074707 	 27	0.4340178072452545 	 34	0.4340567886829376 	 36	0.43404388427734375 	 36	0.4340313971042633 	 39	0.43401989340782166 	 36	0.43403416872024536 	 35	0.43402615189552307 	 21	0.4340381920337677 	 23	0.4340296983718872 	 28	0.43402883410453796 	 32	0.4340256154537201 	 35	0.4340353012084961 	 31	0.43402567505836487 	 30	0.4340396225452423 	 31	0.43403780460357666 	 35	0.43403586745262146 	 35	0.4340536892414093 	 21	0.43402430415153503 	 22	0.4340299665927887 	 28	0.4340410530567169 	 34	0.4340347647666931 	 42	0.43404436111450195 	 26	0.4340305030345917 	 26	0.43402498960494995 	 31	0.4340282380580902 	 41	0.4340481460094452 	 33	0.43403518199920654 	 27	0.43403494358062744 	 32	0.43402865529060364

#### Case 2 (Circle)

In [21]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
#num = 6    # slant p*
num = 7    # circle p*
#num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_mse/set_{}/'.format(num)
mse_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/circle/X_trn.npy')[:N]
y = np.load('data/mvn/circle/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [22]:
p_star = 1.28
params = {'loss': get_mse(p_star), 'd': 2}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(mse_filestr.format(i))
    print()

0	0.3455941379070282 	 53	
1	0.34517067670822144 	 36	
2	0.34552499651908875 	 35	
3	0.3454993963241577 	 39	
4	0.3452836871147156 	 46	
5	0.3453028202056885 	 38	
6	0.3454292416572571 	 31	
7	0.3452165424823761 	 33	
8	0.3452076315879822 	 43	
9	0.34521085023880005 	 29	
10	0.3453024923801422 	 32	
11	0.345602810382843 	 29	
12	0.34551966190338135 	 37	
13	0.3453008234500885 	 39	
14	0.34549036622047424 	 39	
15	0.3453017473220825 	 29	
16	0.34537094831466675 	 31	
17	0.34523212909698486 	 36	
18	0.3451666235923767 	 37	
19	0.34533706307411194 	 42	
20	0.3452540934085846 	 27	
21	0.34520530700683594 	 38	
22	0.34527453780174255 	 46	
23	0.34569036960601807 	 43	
24	0.3452412188053131 	 38	
25	0.3451264798641205 	 34	
26	0.34553322196006775 	 41	
27	0.34522005915641785 	 34	
28	0.345335990190506 	 32	
29	0.34524786472320557 	 41	
30	0.345283180475235 	 39	
31	0.3451896905899048 	 43	
32	0.34529533982276917 	 34	
33	0.3452586829662323 	 44	
34	0.34511470794677734 	 46	
35	0.345144927501

#### Case 3 (Hyperbola)

In [23]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
#num = 6    # slant p*
#num = 7    # circle p*
num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_mse/set_{}/'.format(num)
mse_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/hyperbola/X_trn.npy')[:N]
y = np.load('data/mvn/hyperbola/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [24]:
p_star = -0.44
params = {'loss': get_mse(p_star), 'd': 2}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(mse_filestr.format(i))
    print()

0	1.3324981927871704 	 49	
1	1.3323349952697754 	 37	
2	1.3323843479156494 	 33	
3	1.3324757814407349 	 46	
4	1.3323338031768799 	 37	
5	1.3323932886123657 	 31	
6	1.3323465585708618 	 36	
7	1.3323100805282593 	 32	
8	1.332335352897644 	 36	
9	1.3324286937713623 	 29	
10	1.332427740097046 	 31	
11	1.3323192596435547 	 43	
12	1.3323910236358643 	 38	
13	1.3323655128479004 	 33	
14	1.3323662281036377 	 31	
15	1.3323477506637573 	 34	
16	1.3324052095413208 	 30	
17	1.3324127197265625 	 33	
18	1.3324296474456787 	 45	
19	1.3323816061019897 	 35	
20	1.3324962854385376 	 41	
21	1.332334041595459 	 42	
22	1.3324671983718872 	 41	
23	1.3325074911117554 	 49	
24	1.332441806793213 	 44	
25	1.33235502243042 	 34	
26	1.3325337171554565 	 37	
27	1.332317590713501 	 34	
28	1.332409143447876 	 48	
29	1.3324921131134033 	 44	
30	1.3323370218276978 	 41	
31	1.332596778869629 	 33	
32	1.3323372602462769 	 39	
33	1.332308292388916 	 38	
34	1.3325382471084595 	 40	
35	1.3324589729309082 	 33	
36	1.3323868

#### Case 4 (Checker)

In [25]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
#num = 6    # slant p*
#num = 7    # circle p*
#num = 8    # hyperbola p*
num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_mse/set_{}/'.format(num)
mse_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/checker/X_trn.npy')[:N]
y = np.load('data/mvn/checker/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [26]:
p_star = 1.6
params = {'loss': get_mse(p_star), 'd': 2}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(mse_filestr.format(i))
    print()

0	0.3158361613750458 	 57	
1	0.31584596633911133 	 50	
2	0.3158406913280487 	 46	
3	0.31580328941345215 	 52	
4	0.3158297836780548 	 49	
5	0.31581437587738037 	 43	
6	0.3158365488052368 	 37	
7	0.3158601224422455 	 42	
8	0.31585490703582764 	 47	
9	0.31591176986694336 	 41	
10	0.31584563851356506 	 51	
11	0.3158859312534332 	 37	
12	0.31587332487106323 	 37	
13	0.3158762454986572 	 30	
14	0.315822958946228 	 52	
15	0.3158171772956848 	 44	
16	0.315837025642395 	 50	
17	0.31589552760124207 	 43	
18	0.3158760070800781 	 47	
19	0.31583818793296814 	 35	
20	0.31582963466644287 	 34	
21	0.31585773825645447 	 44	
22	0.3158317506313324 	 34	
23	0.3158170282840729 	 39	
24	0.3158611059188843 	 49	
25	0.31586045026779175 	 42	
26	0.31586357951164246 	 45	
27	0.31585463881492615 	 31	
28	0.3158470392227173 	 36	
29	0.3158455491065979 	 42	
30	0.31587108969688416 	 31	
31	0.3159072697162628 	 40	
32	0.31588271260261536 	 35	
33	0.31584492325782776 	 42	
34	0.31588074564933777 	 47	
35	0.315866053

### $r^*$-SQR Models

#### Case 0 (Vertical)

In [37]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

num = 5    # vertical p*
#num = 6    # slant p*
#num = 7    # circle p*
#num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_sqr/set_{}/'.format(num)
sqr_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/vertical/X_trn.npy')[:N]
y = np.load('data/mvn/vertical/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [38]:
r_star = 0.018
params = {'loss': get_exp_sqr(r_star), 'd': 2, 'output': 'linear'}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(sqr_filestr.format(i))
    print()

0	0.9953616261482239 	 63	
1	0.9953606128692627 	 52	
2	0.9953638911247253 	 66	
3	0.9953690767288208 	 62	
4	0.9953630566596985 	 53	
5	0.9953633546829224 	 49	
6	0.9953652620315552 	 68	
7	0.9953628182411194 	 62	
8	0.9953634738922119 	 53	
9	0.9953649044036865 	 78	
10	0.9953584671020508 	 70	
11	0.9953624606132507 	 78	
12	0.9953637719154358 	 56	
13	0.9953779578208923 	 68	
14	0.9953619241714478 	 57	
15	0.9953617453575134 	 73	
16	0.9953697323799133 	 57	
17	0.9953640103340149 	 67	
18	0.9953641295433044 	 54	
19	0.9953615665435791 	 60	
20	0.9953555464744568 	 71	
21	0.9953605532646179 	 59	
22	0.9953649044036865 	 65	
23	0.9953622221946716 	 54	
24	0.9953626990318298 	 64	
25	0.9953614473342896 	 62	
26	0.9953561425209045 	 92	
27	0.9953652024269104 	 60	
28	0.9953649640083313 	 60	
29	0.9953646659851074 	 50	
30	0.9953680038452148 	 56	
31	0.9953644871711731 	 51	
32	0.9953668117523193 	 54	
33	0.9953611493110657 	 53	
34	0.9953693747520447 	 51	
35	0.9953647255897522 	 48	
36

#### Case 1 (Slant)

In [39]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
num = 6    # slant p*
#num = 7    # circle p*
#num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_sqr/set_{}/'.format(num)
sqr_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/slant/X_trn.npy')[:N]
y = np.load('data/mvn/slant/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [40]:
r_star = 0.018
params = {'loss': get_exp_sqr(r_star), 'd': 2, 'output': 'linear'}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(sqr_filestr.format(i))
    print()

0	0.9951978921890259 	 55	
1	0.9951961636543274 	 77	
2	0.9951959252357483 	 49	
3	0.995198130607605 	 42	
4	0.9952021241188049 	 54	
5	0.9951931238174438 	 46	
6	0.9951971769332886 	 40	
7	0.9951969981193542 	 46	
8	0.9951976537704468 	 54	
9	0.9951919913291931 	 49	
10	0.9951960444450378 	 45	
11	0.9951937794685364 	 46	
12	0.9951995611190796 	 54	
13	0.995197057723999 	 53	
14	0.9951983690261841 	 61	
15	0.9951967000961304 	 57	
16	0.9951977729797363 	 50	
17	0.9951971173286438 	 53	
18	0.9951989650726318 	 59	
19	0.9951964616775513 	 43	
20	0.9951989650726318 	 50	
21	0.9951992034912109 	 50	
22	0.9951962828636169 	 47	
23	0.9951951503753662 	 44	
24	0.9952007532119751 	 51	
25	0.9951973557472229 	 42	
26	0.9951993823051453 	 58	
27	0.9952007532119751 	 43	
28	0.9951959848403931 	 39	
29	0.9951947927474976 	 45	
30	0.9951984882354736 	 52	
31	0.9951966404914856 	 35	
32	0.9951932430267334 	 58	
33	0.995194137096405 	 52	
34	0.9951915144920349 	 48	
35	0.9951969385147095 	 56	
36	0.

#### Case 2 (Circle)

In [41]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
#num = 6    # slant p*
num = 7    # circle p*
#num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_sqr/set_{}/'.format(num)
sqr_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/circle/X_trn.npy')[:N]
y = np.load('data/mvn/circle/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [42]:
r_star = -0.1
params = {'loss': get_exp_sqr(r_star), 'd': 2, 'output': 'linear'}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(sqr_filestr.format(i))
    print()

0	0.9398018717765808 	 45	
1	0.9398988485336304 	 48	
2	0.939784049987793 	 44	
3	0.9397011995315552 	 47	
4	0.9396071434020996 	 50	
5	0.9402419924736023 	 48	
6	0.9397205710411072 	 59	
7	0.9397492408752441 	 41	
8	0.9396451115608215 	 60	
9	0.9396093487739563 	 54	
10	0.9395895600318909 	 60	
11	0.9397324919700623 	 45	
12	0.9396385550498962 	 60	
13	0.9399023652076721 	 53	
14	0.9398393630981445 	 57	
15	0.9395857453346252 	 60	
16	0.9395516514778137 	 60	
17	0.939750075340271 	 51	
18	nan 	 60	
19	0.9399604201316833 	 61	
20	nan 	 55	
21	0.9397198557853699 	 45	
22	0.9399459958076477 	 53	
23	0.939701497554779 	 62	
24	nan 	 67	
25	0.9396522641181946 	 43	
26	0.9401387572288513 	 42	
27	0.9397886395454407 	 46	
28	0.9396680593490601 	 51	
29	0.939606249332428 	 58	
30	0.9396438598632812 	 49	
31	0.9398946166038513 	 50	
32	nan 	 61	
33	0.9396752715110779 	 51	
34	nan 	 50	
35	nan 	 49	
36	0.9396519660949707 	 56	
37	0.939643144607544 	 48	
38	0.9396164417266846 	 48	
39	0.93954211

#### Case 3 (Hyperbola)

In [43]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
#num = 6    # slant p*
#num = 7    # circle p*
num = 8    # hyperbola p*
#num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_sqr/set_{}/'.format(num)
sqr_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/hyperbola/X_trn.npy')[:N]
y = np.load('data/mvn/hyperbola/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [44]:
r_star = -0.2
params = {'loss': get_exp_sqr(r_star), 'd': 2, 'output': 'linear'}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(sqr_filestr.format(i))
    print()

0	0.9399952292442322 	 73	
1	0.9399228692054749 	 85	
2	0.9399579167366028 	 84	
3	0.9399397373199463 	 71	
4	0.9399492144584656 	 85	
5	0.9399340152740479 	 100	
6	0.9399597644805908 	 87	
7	0.939960777759552 	 91	
8	0.9399221539497375 	 83	
9	0.9399409890174866 	 84	
10	0.9399415254592896 	 91	
11	0.9399675130844116 	 82	
12	0.9399179816246033 	 100	
13	0.9399842619895935 	 79	
14	0.9400355219841003 	 64	
15	0.939929723739624 	 85	
16	0.9399088621139526 	 100	
17	0.9399646520614624 	 87	
18	0.9399712085723877 	 93	
19	0.9399605393409729 	 99	
20	0.939885139465332 	 91	
21	0.9399747848510742 	 86	
22	0.9399537444114685 	 93	
23	0.939976692199707 	 85	
24	0.9399703145027161 	 72	
25	0.939932644367218 	 96	
26	0.9400007724761963 	 100	
27	0.9399153590202332 	 91	
28	0.9399386048316956 	 93	
29	0.9399744868278503 	 74	
30	0.9399392604827881 	 84	
31	0.939966082572937 	 88	
32	0.9399676322937012 	 100	
33	0.9399296045303345 	 91	
34	0.9399669170379639 	 87	
35	0.9399557709693909 	 97	
36	

#### Case 4 (Checker)

In [45]:
# Multivariate
#num = 0    # vertical
#num = 1    # slant
#num = 2    # circle
#num = 3    # hyperbola
#num = 4    # checker

#num = 5    # vertical p*
#num = 6    # slant p*
#num = 7    # circle p*
#num = 8    # hyperbola p*
num = 9    # checker p*
reps = 100

# File parameters
filestr = 'models/multivariate/ab_sqr/set_{}/'.format(num)
sqr_filestr = filestr + 'model_{}.h5'

if not os.path.isdir(filestr):
    os.mkdir(filestr)

# Data parameters
N = 10**6
X = np.load('data/mvn/checker/X_trn.npy')[:N]
y = np.load('data/mvn/checker/y_trn.npy')[:N].astype('float32')
data, m, s = split_data(X, y)

In [46]:
r_star = -0.1
params = {'loss': get_exp_sqr(r_star), 'd': 2, 'output': 'linear'}
for i in range(reps):
    print(i, end = '\t')
    model, trace = train(data, **params)
    model.save_weights(sqr_filestr.format(i))
    print()

0	0.9817698001861572 	 100	
1	0.9817615151405334 	 92	
2	0.9817500710487366 	 95	
3	0.9817914366722107 	 73	
4	0.981763482093811 	 90	
5	0.9817686080932617 	 77	
6	0.9817808866500854 	 84	
7	0.9817582368850708 	 83	
8	0.9817496538162231 	 82	
9	0.9817550778388977 	 74	
10	0.9817600846290588 	 78	
11	0.9817867875099182 	 86	
12	0.9817458391189575 	 100	
13	0.9817692637443542 	 88	
14	0.9818233847618103 	 66	
15	0.981767475605011 	 100	
16	0.981781005859375 	 75	
17	0.981785774230957 	 81	
18	0.9818099737167358 	 88	
19	0.9817842841148376 	 84	
20	0.9818118214607239 	 73	
21	0.9817802309989929 	 80	
22	0.981795072555542 	 83	
23	0.9817672967910767 	 92	
24	0.981812596321106 	 100	
25	0.9817992448806763 	 65	
26	0.9817826151847839 	 94	
27	0.9817687273025513 	 72	
28	0.9817553162574768 	 82	
29	0.9817668199539185 	 85	
30	0.9817823767662048 	 86	
31	0.9817469716072083 	 100	
32	0.9817742705345154 	 100	
33	0.9817743897438049 	 82	
34	0.9817897081375122 	 76	
35	0.9817855954170227 	 88	
36